[View in Colaboratory](https://colab.research.google.com/github/hlecuanda/jupyter-notebooks-of-all-kinds/blob/devel/CalendarEvents.ipynb)

##Auth

In [0]:
from google.colab import auth
from googleapiclient.discovery import build
from apiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account
import datetime

try:
    from google.cloud import storage
    from googleapiclient import discovery
    
except:
    !pip install --upgrade google-cloud-storage google-api-client
    from google.cloud import storage
    from googleapiclient import discovery

def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""        
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print('Blob {} downloaded to {}.'.format(
        source_blob_name,
        destination_file_name))

def reset_auth():
    !rm -v = *.json
    
STORAGE_BUCKET="archive.h-lo.me"
STORAGE_PATH = "credentials/"
PROJECT="colab-datalab-nbooks"
SA_FILE="colab-datalab-nbooks-0b23b6ee2449.json"

auth.authenticate_user()
storage_client = storage.Client(PROJECT)       

SCOPES = ['https://www.googleapis.com/auth/calendar']

!rm '$SA_FILE'

download_blob(STORAGE_BUCKET, STORAGE_PATH+SA_FILE, SA_FILE)

credentials = service_account.Credentials.from_service_account_file(
        SA_FILE, scopes=SCOPES).with_subject('hector@lecuanda.com')

# drive  = discovery.build('drive', 'v3', credentials=credentials)
# sheets = discovery.build('sheets','v4', credentials=credentials)
calendar = discovery.build('calendar','v3', credentials=credentials)

## Calendar

In [0]:
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
hago= datetime.datetime(tsnow.year,
                        tsnow.month,
                        1,
                        0).isoformat() + 'Z'

CAL='lecuanda.com_oigsd97meblvho2lcq4fhdvelg@group.calendar.google.com'

events_result = calendar.events().list(calendarId=CAL, 
                                       timeMin=hago,
                                     # maxResults=10, 
                                       singleEvents=True,
                                       orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])
